# 📕 MongoDB with Python: Full Tutorial in Jupyter Notebook
This notebook demonstrates how to:
- Connect to MongoDB
- Create collections and insert synthetic data
- Perform CRUD operations
- Use `$lookup` for JOIN-like behavior
- Use `$group` for aggregation (COUNT, SUM, AVG, MAX, MIN)


In [1]:
# ✅ Install PyMongo
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 855.2/855.2 kB 1.2 MB/s eta 0:00:00a 0:00:01


In [15]:
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
print(client.list_database_names())

['admin', 'config', 'local']


In [16]:
from pymongo import MongoClient

# Connect to MongoDB (Make sure MongoDB is running locally or adjust host if needed)
client = MongoClient('mongodb://localhost:27017/')

# Create the database and collections
db = client['ShopDB']  # Create a database named 'ShopDB'
customers = db['customers']  # Create 'customers' collection
orders = db['orders']  # Create 'orders' collection

# 🧹 Clean up any existing data
customers.delete_many({})
orders.delete_many({})

# 🧪 Insert synthetic data into 'customers' collection
customers.insert_many([
    {"_id": 1, "name": "Alice"},
    {"_id": 2, "name": "Bob"},
    {"_id": 3, "name": "Charlie"},
    {"_id": 4, "name": "David"}
])

# 🧪 Insert synthetic data into 'orders' collection
orders.insert_many([
    {"_id": 101, "customer_id": 1, "amount": 250.00},
    {"_id": 102, "customer_id": 1, "amount": 150.50},
    {"_id": 103, "customer_id": 2, "amount": 450.75},
    {"_id": 104, "customer_id": 3, "amount": 300.00},
    {"_id": 105, "customer_id": 2, "amount": 120.25}
])

# Check if data is inserted
print("Customers Data:", list(customers.find()))
print("Orders Data:", list(orders.find()))

Customers Data: [{'_id': 1, 'name': 'Alice'}, {'_id': 2, 'name': 'Bob'}, {'_id': 3, 'name': 'Charlie'}, {'_id': 4, 'name': 'David'}]
Orders Data: [{'_id': 101, 'customer_id': 1, 'amount': 250.0}, {'_id': 102, 'customer_id': 1, 'amount': 150.5}, {'_id': 103, 'customer_id': 2, 'amount': 450.75}, {'_id': 104, 'customer_id': 3, 'amount': 300.0}, {'_id': 105, 'customer_id': 2, 'amount': 120.25}]


In [17]:
from pymongo import MongoClient

# Connect to MongoDB (adjust host if needed)
client = MongoClient('mongodb://localhost:27017/')
db = client['ShopDB']
customers = db['customers']
orders = db['orders']

In [18]:
from pymongo import MongoClient

# Connect to MongoDB (adjust host if needed)
client = MongoClient('mongodb://localhost:27017/')
db = client['ShopDB']
customers = db['customers']
orders = db['orders']

# 🧹 Clean up any existing data
customers.delete_many({})
orders.delete_many({})

# 🧪 Insert Synthetic Data
customers.insert_many([
    {"_id": 1, "name": "Alice"},
    {"_id": 2, "name": "Bob"},
    {"_id": 3, "name": "Charlie"}
])

orders.insert_many([
    {"_id": 101, "customer_id": 1, "amount": 250.00},
    {"_id": 102, "customer_id": 1, "amount": 150.50},
    {"_id": 103, "customer_id": 2, "amount": 450.75},
    {"_id": 104, "customer_id": 3, "amount": 300.00},
    {"_id": 105, "customer_id": 2, "amount": 120.25}
])

# 🧑‍💻 CRUD Operations in MongoDB
# Create: Insert Data
customers.insert_one({"_id": 4, "name": "David"})
orders.insert_one({"_id": 106, "customer_id": 4, "amount": 200.50})

# Read: Find a customer
customer = customers.find_one({"_id": 4})
print(customer)

# Update: Change a customer's name
customers.update_one({"_id": 4}, {"$set": {"name": "David Updated"}})
updated_customer = customers.find_one({"_id": 4})
print(updated_customer)

# Delete: Remove a customer and their orders
orders.delete_many({"customer_id": 4})
customers.delete_one({"_id": 4})
deleted_customer = customers.find_one({"_id": 4})
print(deleted_customer)  # Should return None

{'_id': 4, 'name': 'David'}
{'_id': 4, 'name': 'David Updated'}
None


In [19]:
# 🔍 $lookup (JOIN-like behavior)
pipeline = [
    {"$lookup": {
        "from": "customers",
        "localField": "customer_id",
        "foreignField": "_id",
        "as": "customer_info"
    }},
    {"$unwind": "$customer_info"},
    {"$project": {
        "_id": 0,
        "customer": "$customer_info.name",
        "amount": 1
    }}
]
results = orders.aggregate(pipeline)
for row in results:
    print(row)

{'amount': 250.0, 'customer': 'Alice'}
{'amount': 150.5, 'customer': 'Alice'}
{'amount': 450.75, 'customer': 'Bob'}
{'amount': 300.0, 'customer': 'Charlie'}
{'amount': 120.25, 'customer': 'Bob'}


In [20]:
# 📊 Group by customer_id with aggregate functions
pipeline = [
    {
        "$group": {
            "_id": "$customer_id",
            "order_count": {"$sum": 1},
            "total_amount": {"$sum": "$amount"},
            "avg_amount": {"$avg": "$amount"},
            "max_amount": {"$max": "$amount"},
            "min_amount": {"$min": "$amount"}
        }
    }
]
results = orders.aggregate(pipeline)
for row in results:
    print(row)

{'_id': 1, 'order_count': 2, 'total_amount': 400.5, 'avg_amount': 200.25, 'max_amount': 250.0, 'min_amount': 150.5}
{'_id': 2, 'order_count': 2, 'total_amount': 571.0, 'avg_amount': 285.5, 'max_amount': 450.75, 'min_amount': 120.25}
{'_id': 3, 'order_count': 1, 'total_amount': 300.0, 'avg_amount': 300.0, 'max_amount': 300.0, 'min_amount': 300.0}
